# ModifyInstance

<a href="#" onclick="history.back()">&lt;--- Back</a>

Existing CIM instances can be modified by having the values of properties changed. This is achieved using the [`ModifyInstance()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.ModifyInstance) method. It takes a [`pywbem.CIMInstance`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.CIMInstance) object as input, which references the CIM instance to be modified with its `path` attribute, and specifies new values for the properties.

The `PropertyList` input parameter of the method specifies the names of the properties that are to be modified. If this parameter is not provided, all properties are modified. Those properties that are to be modified but have no new values specified in the input instance get their default values.

The values of key properties cannot be modified.

Again, the CIM provider on the WBEM server may or may not support the modification of CIM instances.

In [ ]:
from __future__ import print_function
import sys
import pywbem

username = 'user'
password = 'password'
classname = 'CIM_ComputerSystem'
namespace = 'root/interop'
server = 'http://localhost'

conn = pywbem.WBEMConnection(server, (username, password),
                             default_namespace=namespace,
                             no_verification=True)

filter_inst = pywbem.CIMInstance(
    'CIM_IndicationFilter',
    {'Name': 'pywbem_test',
     'Query': 'SELECT * FROM CIM_Indication',
     'QueryLanguage': 'WQL'})
    
print('Creating instance of class: %s' % filter_inst.classname)
try:
    filter_path = conn.CreateInstance(filter_inst, namespace)
except pywbem.Error as exc:
    if isinstance(exc, pywbem.CIMError) and \
       exc.status_code == pywbem.CIM_ERR_NOT_SUPPORTED:
        print('WBEM server does not support creation of dynamic filters.')
        filter_path = None
    else:
        print('CreateInstance failed: %s: %s' % (exc.__class__.__name__, exc))
        sys.exit(1)

if filter_path is not None:
    print('Created instance: %s' % filter_path)

    filter_inst['Query'] = 'SELECT * FROM CIM_ProcessIndication'

    filter_inst.path = filter_path

    print('Modifying the instance')
    try:
        conn.ModifyInstance(filter_inst, PropertyList=['Query'])
    except pywbem.Error as exc:
        if isinstance(exc, pywbem.CIMError) and exc[0] == pywbem.CIM_ERR_NOT_SUPPORTED:
            print('Modifying CIM_IndicationFilter is not supported')
        else:
            print('ModifyInstance failed: %s: %s' % (exc.__class__.__name__, exc))
            sys.exit(1)

    print('Deleting the instance again, to clean up')
    try:
        conn.DeleteInstance(filter_path)
    except pywbem.Error as exc:
        print('DeleteInstance failed: %s: %s' % (exc.__class__.__name__, exc))
        sys.exit(1)
    print('Deleted the instance')

The `ModifyInstance()` method takes the instance path from the `path` attribute of its instance argument. Because the instance path of the new instance is only known after `CreateInstance()` returns successfully, we need to set the `path` attribute of the instance object accordingly.

<a href="#" onclick="history.back()">&lt;--- Back</a>